# Assignment 3

This structured output and tools uses OpenBreweryDB endpoint: https://api.openbrewerydb.org/v1/breweries to find breweries in a given location and outputs them and/or "saves" them for later.


Install openai dependency (used a virtual environment)
AI helped with confusing python installation locations, look at note.md for reference

In [8]:
from openai import OpenAI
from typing import Literal, List
import json
import requests
import os
from dotenv import load_dotenv

load_dotenv()
token_one = os.getenv("OPENAI_KEY")
client = OpenAI(api_key=token_one)

# Reference
"""
    OpenBreweryDB endpoint:
      https://api.openbrewerydb.org/v1/breweries

    Returns:
      List of breweries in a given city or state
"""
def get_breweries(
    city: str,
    state: str = None,
    limit: int = 3,
    debug: bool = False,
) -> dict:
    url = "https://api.openbrewerydb.org/v1/breweries"
    params = {
        "by_city": city,
        "per_page": limit,
    }

    if state:
        params["by_state"] = state

    r = requests.get(url, params=params, timeout=30)
    r.raise_for_status()
    data = r.json()

    breweries = []
    for b in data:
        breweries.append({
            "name": b.get("name"),
            "brewery_type": b.get("brewery_type"),
            "address": ", ".join(
                filter(None, [
                    b.get("street"),
                    b.get("city"),
                    b.get("state"),
                ])
            ),
            "website": b.get("website_url"),
        })

    out = {
        "city": city,
        "state": state,
        "count": len(breweries),
        "breweries": breweries,
        "source": "openbrewerydb.org",
    }

    if debug:
        out["_debug"] = {"request_url": r.url}

    return out


def save_recommendation(title: str, notes: str, people: List[str]) -> dict:
    return {
        "status": "saved",
        "title": title,
        "notes": notes,
        "shared_with": people,
    }

# Tools
'''Gets the breweries based on input, and/or saves a bewery recommendation'''
tools = [
    {
        "type": "function",
        "name": "get_breweries",
        "description": "Find breweries in a given city using OpenBreweryDB.",
        "parameters": {
            "type": "object",
            "properties": {
                "city": {"type": "string", "description": "City name, e.g. 'Durham'"},
                "state": {"type": "string", "description": "State name, e.g. 'North Carolina'"},
                "limit": {"type": "integer", "description": "Max number of breweries"},
            },
            "required": ["city"],
            "additionalProperties": False,
        },
    },
    {
        "type": "function",
        "name": "save_recommendation",
        "description": "Save a recommendation or plan for later reference.",
        "parameters": {
            "type": "object",
            "properties": {
                "title": {"type": "string"},
                "notes": {"type": "string"},
                "people": {"type": "array", "items": {"type": "string"}},
            },
            "required": ["title", "notes", "people"],
            "additionalProperties": False,
        },
    },
]

prompt = """
Argyle is a brewery owner who wants to know the competition in my area. 
What are some notable breweries in the Durham, NC area?
"""

input_list = [{"role": "user", "content": prompt}]

resp = client.responses.create(
    model="gpt-4o-mini-2024-07-18",
    tools=tools,
    tool_choice="auto",
    input=input_list,
)

print(resp)

input_list += resp.output

for item in resp.output:
    if item.type == "function_call":
        args = json.loads(item.arguments)
        print("MODEL CHOSE TOOL:", item.name)
        print("ARGS:", args)

        if item.name == "get_breweries":
            result = get_breweries(**args)
        elif item.name == "save_recommendation":
            result = save_recommendation(**args)
        else:
            raise ValueError(f"Unknown tool: {item.name}")

        input_list.append(
            {
                "type": "function_call_output",
                "call_id": item.call_id,
                "output": json.dumps(result),
            }
        )

final = client.responses.create(
    model="gpt-4o-mini-2024-07-18",
    tools=tools,
    input=input_list,
)

print("\nFINAL OUTPUT:")
print(final.output_text)


Response(id='resp_0e969e1220b998f900697fcd49dfb4819d9aa6ddaf11ab4da7', created_at=1769983305.0, error=None, incomplete_details=None, instructions=None, metadata={}, model='gpt-4o-mini-2024-07-18', object='response', output=[ResponseFunctionToolCall(arguments='{"city":"Durham","state":"North Carolina","limit":5}', call_id='call_s8WNg41dW5KPOICD68JPpl82', name='get_breweries', type='function_call', id='fc_0e969e1220b998f900697fcd4ab878819d843be435c67f8985', status='completed')], parallel_tool_calls=True, temperature=1.0, tool_choice='auto', tools=[FunctionTool(name='get_breweries', parameters={'type': 'object', 'properties': {'city': {'type': 'string', 'description': "City name, e.g. 'Durham'"}, 'state': {'type': 'string', 'description': "State name, e.g. 'North Carolina'"}, 'limit': {'type': 'integer', 'description': 'Max number of breweries'}}, 'required': ['city', 'state', 'limit'], 'additionalProperties': False}, strict=True, type='function', description='Find breweries in a given ci